## Tarea 2 Conteo aproximado con MCMC aplicado a los modelos Hard-Core y q-coloraciones

En esta tarea se explora el método MCMC para realizar conteos aproximados de las configuraciones factibles en Hardcore y
las q-coloraciones posibles en un grafo. Ambos en grafos $KxK$.

### Conteo aproximado q-coloraciones

### Conteo aproximado config factibles hard-core

In [ ]:
import copy

import utils.gibbs_sampler as gs

In [ ]:
K = 5

zero_grid, ones_grid = gs.create_k_matrix(5)
# Right now no connection between nodes
adjacency_matrix = copy.deepcopy(zero_grid)




